In [3]:
# Extracción
import pandas as pd
import os
import re
import glob

# Transformación
from FuncionesETL_simple.funciones_ETL_simple import *

# Carga
import psycopg2
import sqlalchemy as db

## - EXTRACCIÓN

In [64]:
folder_path = os.getcwd() + "\\Datasets\\" # Ruta de la carpeta donde están los archivos
file_path = glob.glob(folder_path + "*") # Lista de rutas de los archivos de precios

In [65]:
# Bucle que recorre todos los archivos de la carpeta, crea un df y lo agrega a una lista
lst_df = []

for path in file_path:
    df = importar(path)  # La función importar se encuentra en el archivo funciones_ETL.py
    lst_df.append(df)

In [66]:
# Lista con los nombres de archivos
lst_keys = []
for path in file_path:
    filename = os.path.split(path)[1]
    filename_mod = filename.split('.')[0]
    lst_keys.append(filename_mod)

In [67]:
# Diccionario que contiene cada df y le asigna como key el nombre del archivo correspondiente
dicc_df={}
for e in range(len(lst_keys)):
    key = lst_keys[e]
    df = lst_df[e] 
    dicc_df[key] = df

---

## - TRANSFORMACIÓN

In [68]:
transformacion(dicc_df) # ver función en archivo funciones_ETL.py

{'precios_semanas_20200419_20200426':         precio    producto_id sucursal_id fecha_semana
 0        198.9  9990385651946    5-1-2003   2020-04-19
 1        349.0  7790520000064      50-1-2   2020-04-19
 2        449.0  7790520000071      49-1-2   2020-04-19
 3        449.0  7790520000071      50-1-2   2020-04-19
 4        122.5  7790520000125   20-1-2001   2020-04-19
 ...        ...            ...         ...          ...
 937447   830.0  7792410010047     10-1-46   2020-04-26
 937448   930.0  7792410010047     10-1-48   2020-04-26
 937449  1030.0  7792410010047      10-1-6   2020-04-26
 937450   980.0  7792410010047    10-3-465   2020-04-26
 937451  4732.0  7899026484603   11-5-2997   2020-04-26
 
 [920483 rows x 4 columns],
 'precios_semana_20200413':         precio    producto_id sucursal_id fecha_semana
 0        29.90  0000000001663     2-1-014   2020-04-13
 1        29.90  0000000002288     2-1-032   2020-04-13
 2        39.90  0000000002288     2-1-096   2020-04-13
 3       4

---

## - CARGA

In [70]:
from conexion_DB import *

In [71]:
dicc_df['producto'].to_sql(name="producto", con = conection, if_exists='append', index=False)

941

In [72]:
dicc_df['sucursal'].to_sql(name="sucursal", con = conection, if_exists='append', index=False)

333

In [75]:
lst_df = []
for key in dicc_df.keys():
    if 'precios_semana' in key:
        lst_df.append(dicc_df[key])

tabla_precio = pd.concat(lst_df)

tabla_precio = tabla_precio.sort_values('fecha_semana')

tabla_precio.to_sql(name="precio", con = conection, if_exists='append', index=False)

351